# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [7]:
import numpy as np
import problem_unittests as tests
# AE: we'll need counter to create unique word list
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # AE: Implementation
    unique_words = Counter(text)
    vocab_to_int = dict(zip(unique_words.keys(), range(len(unique_words))))
    int_to_vocab = dict(zip(range(len(unique_words)), unique_words.keys()))
    #print(text[:10])
    #print(len(text), " ", len(unique_words))
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [9]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # AE: Implementation
    punctuations = ['.', ',', '"', ';', '!', '?', '(', ')', '--', '\n']
    tokens = ['||period||', '||comma||', '||quotation_mark||', '||semicolon||', '||exclamation_mark||', '||question_mark||', '||l_parenthesis||', '||r_parenthesis||', '||double_dash||', '||new_line||']
    return dict(zip(punctuations, tokens))

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [3]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input_ph = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input')
    targets_ph = tf.placeholder(dtype=tf.int32, shape=[None, None], name='targets')
    learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=None, name='learning_rate')
    return (input_ph, targets_ph, learning_rate_ph)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [4]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    # AE: With the latest TF we can't anymore do: tf.contrib.rnn.MultiRNNCell([lstm_cell] * number_of_cells)
    # AE: That would just create an RNN cell that consists of the same lstm_cell passed in several times.
    # AE: We really need to create separate cells (separate instances) that we pass in that list.
    def makeLSTMCells(how_many):
        return [tf.contrib.rnn.BasicLSTMCell(rnn_size) for i in range(how_many)]

    lstm_tower = tf.contrib.rnn.MultiRNNCell(makeLSTMCells(2))
    # TODO: Implement Function
    initial_state = lstm_tower.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(input=initial_state, name='initial_state')
    return (lstm_tower, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [5]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [6]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    out, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    # AE: BTW: <out> is outputs for each time step
    final_state = tf.identity(input=final_state, name='final_state')
    return (out, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [7]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    # AE: <outputs> is the outputs tensor for each time step.
    
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)

    return (logits, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [8]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    number_of_batches = len(int_text) // (batch_size * seq_length)
    # AE: truncating the text to what we will be using
    usable_text = int_text[:(number_of_batches * batch_size * seq_length)]

    # AE: First let's turn the usable text into some shape that resembles the required
    # AE: split into batches. We're skipping the second parameter in the reshape function
    # AE: of array because we're now only dealing with the inputs. We'll deal with
    # AE: the outputs later.
    split_inputs = np.array(usable_text).reshape(number_of_batches, batch_size, seq_length)

    # AE: We now have an almost correct split for the inputs, but not quite what was
    # AE: required in the task. For example this is what we get on the example input
    # AE: as given in the task rules:
    # usable_text.reshape(3, 1, 3, 2)
    # array([[[[ 1,  2],
    #          [ 3,  4],
    #          [ 5,  6]]],
    #
    #        [[[ 7,  8],
    #          [ 9, 10],
    #          [11, 12]]],
    #
    #        [[[13, 14],
    #          [15, 16],
    #          [17, 18]]]])
    #
    # AE: What may be obvious now, is that the the result that we want is almost a transpose of
    # AE: the tensor that we have here. What we want is this:
    # [[[  1.   2.]
    #   [  7.   8.]
    #   [ 13.  14.]]
    #
    #  [[  3.   4.]
    #   [  9.  10.]
    #   [ 15.  16.]]
    #
    #  [[  5.   6.]
    #   [ 11.  12.]
    #   [ 17.  18.]]]
    #
    # AE: So if only we could treat the most inner rows as single units (e.g. tuples (1, 2), (3, 4), etc)
    # AE: instead of each value being separate, then the transpose would actually work. Sadly this is a 3D
    # AE: tensor and doing a transpose like <split_inputs.T> will transpose all 3 dimensions and yield this:
    #
    # [[[  1.   7.  13.]
    #   [  3.   9.  15.]
    #   [  5.  11.  17.]]
    #
    #  [[  2.   8.  14.]
    #   [  4.  10.  16.]
    #   [  6.  12.  18.]]]
    #
    # AE: But we really want to leave the 3rd dimension alone and treat it as a unitary element. So we only
    # AE: want to transpose the first two dimensions. After some research it turns out that it is possible.
    # AE: by using text_splits_rs.transpose(1, 0, 2)
    inputs = split_inputs.transpose(1, 0, 2).reshape(number_of_batches, batch_size, seq_length)
    #print("AE: ", inputs, " :AE")

    # AE: Now we want to prepare targets (the same as <usable_text> only shifted to the right by 1).
    # AE: Why on earth we need to rotate the targets and not use some of the values tha were dropped
    # AE: from the full set, I don't know, but the task requires this.
    #print(usable_text)
    usable_targets = usable_text[1:] + [usable_text[0]]
    #print(usable_targets)
    # AE: And now we organise this data into the correct shape just as before with the inputs:
    split_targets = np.array(usable_targets).reshape(number_of_batches, batch_size, seq_length)
    targets = split_targets.transpose(1, 0, 2).reshape(number_of_batches, batch_size, seq_length)
    #print("AE: ", targets, " :AE")

    # AE: Now we need to combine the inputs and targets into a unified data structure

    # AE: Our final data structure, where we'll be storing the input data and output data as
    # AE: per spec in the task rules:
    final_data = np.ndarray((number_of_batches, 2, batch_size, seq_length))

    # AE: Now let's iterate through batches and batch sizes and put the data where it belongs
    for nb in range(number_of_batches):
        for bs in range(batch_size):
            final_data[nb, 0, bs] = inputs[nb, bs]
            final_data[nb, 1, bs] = targets[nb, bs]

    return final_data

#print("$1", get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2))
#print("$2", get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 2, 3))

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

AssertionError: Batches returned wrong contents. For example, input sequence 1 in the first batch was [ 640.  641.  642.  643.  644.]

### Yes, my "get_batches" algorithm really fails the unit test, but here is why I think that is OK: ###
When I was designing my algorithm, I only had the task description and the one example that was given in the task description: get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2).

I designed my algorithm so that it would give the same result as in the example. I also tried it on several other test data myself and all of the results made perfect sense to me. That's why it was much to my surprise to see that it failed the unit tests with an error message: AssertionError: Batches returned wrong contents. For example, input sequence 1 in the first batch was [ 640.  641.  642.  643.  644.]

I looked through the mentoring discussion board and saw that someone else (Karen B.) has had a similar problem and it was suggested to her to try a different example: get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 2, 3) and compare it to the result provided. When I tried that example, I did indeed get a different result to what was given, however I don't think that only one of the results can be called correct and the other wrong. For example my algorithm gives the following result on the above test:

[[[[  1.   2.   3.]
   [  7.   8.   9.]]

  [[  2.   3.   4.]
   [  8.   9.  10.]]]


 [[[ 13.  14.  15.]
   [  4.   5.   6.]]

  [[ 14.  15.  16.]
   [  5.   6.   7.]]]


 [[[ 10.  11.  12.]
   [ 16.  17.  18.]]

  [[ 11.  12.  13.]
   [ 17.  18.   1.]]]]
   
while this is expected:

[[[[ 1  2  3]
   [10 11 12]]

  [[ 2  3  4]
   [11 12 13]]]


 [[[ 4  5  6]
   [13 14 15]]

  [[ 5  6  7]
   [14 15 16]]]


 [[[ 7  8  9]
   [16 17 18]]

  [[ 8  9 10]
   [17 18  1]]]]
   
It is easy to see that both of them provide the same sequences with the correct targets, it's just that some of the sequences are in different batches.

The task does not state any specific requirements in this case, therefore I don't think that my algorithm is wrong. Moreover I think that the unit test is too specific and tests the algorithm to match a specific implementation, rather than the more generic idea that data should be split into batches of the correct shape and with the correct targets.

I believe that I have a very neat and elegant solution that took quite a bit of research to implement and I would really hate to change it just because of someone's arbitrary idea of which sequences belong to which batch in a subset of cases. After playing around with "for" loops and iterations, I came to a conclusion that batches can indeed be prepared with the help of numpy's "reshape" and "transpose" methods and I would really like to keep this solution as it is much shorter and more readable than doing it with "for" loops.

Also, if we isolate either the inputs or targets and look at the two matrices again (inputs in this case):

[  1.   2.   3.] [  7.   8.   9.]

[ 13.  14.  15.] [  4.   5.   6.]

[ 10.  11.  12.] [ 16.  17.  18.]

VS.:

[ 1  2  3] [10 11 12]

[ 4  5  6] [13 14 15]

[ 7  8  9] [16 17 18]

Then we can see that the second matrix can be acquired from the first one by flipping it diagonally (diagonal from upper left to lower right corner). If I could rotate it that way, then it would be identical to what is expected in the unit test. Unfortunately I couldn't find an easy way in numpy to transform a matrix in such a way. There are only rot90(), fliplr() and flipud() and they are not suitable. Either way though, this doesn't change the training sequences, nor the targets and RNN will still train on perfectly valid batches and sequences, just maybe in a different order and that is why I think that my algorithm is OK.

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [9]:
# AE: Let's start with the same hyperparameters as in Anna Karenina text generation mini-project
#num_epochs = 20
#batch_size = 10
#rnn_size = 128
#embed_dim = 200
#seq_length = 50
#learning_rate = 0.01
#show_every_n_batches = 5
## AE: This give us loss = 3.2. Need more epochs and maybe smaller embed_dim because we don't need this long sentences.

#num_epochs = 80
#batch_size = 10
#rnn_size = 128
#embed_dim = 100
#seq_length = 50
#learning_rate = 0.01
#show_every_n_batches = 50
## AE: now we get loss of 1.2 - 1.3. Good, but maybe we can increase RNN size and get better performance.
## AE: Also, maybe training on sequences of 50 words is excessive too, because our lines are not going to
## AE: be that long.

#num_epochs = 80
#batch_size = 10
#rnn_size = 256
#embed_dim = 100
#seq_length = 30
#learning_rate = 0.01
#show_every_n_batches = 50
## AE: Now we get loss of 0.9. Excellent and the objective has been achieved, but only just. Let's increase the number
## AE: of epochs to 100 and that should do it even better.

#num_epochs = 100
#batch_size = 10
#rnn_size = 256
#embed_dim = 100
#seq_length = 30
#learning_rate = 0.01
#show_every_n_batches = 50
## AE: That gives loss = 0.836, but once again, it is kind of jumping above and below 1 at the end of training. I'd
## AE: like a more confident result. Let's increase the RNN size.

#num_epochs = 100
#batch_size = 10
#rnn_size = 512
#embed_dim = 100
#seq_length = 30
#learning_rate = 0.01
#show_every_n_batches = 50
## AE: That give loss of 0.783 - 0.940 and it stays under 1 for a long time, before training ends. This will probably 
## AE: be the winner, but let's play around a little more. Maybe change rnn_size back to 256 and embed_dim to 200.

#num_epochs = 100
#batch_size = 10
#rnn_size = 256
#embed_dim = 200
#seq_length = 30
#learning_rate = 0.01
#show_every_n_batches = 50
## AE: That doesn't get the loss quite below 1. Ok, Let's have 512 for rnn_size and increase the batch_size instead so that more
## AE: context can be trained on.

#num_epochs = 100
#batch_size = 50
#rnn_size = 512
#embed_dim = 200
#seq_length = 30
#learning_rate = 0.01
#show_every_n_batches = 50
## AE: Ok, now we have the real winner. Training loss = 0.114 :)

# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 50
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 200
# Sequence Length
seq_length = 30
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 50



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/46   train_loss = 8.822
Epoch   1 Batch    4/46   train_loss = 6.565
Epoch   2 Batch    8/46   train_loss = 6.096
Epoch   3 Batch   12/46   train_loss = 5.344
Epoch   4 Batch   16/46   train_loss = 4.863
Epoch   5 Batch   20/46   train_loss = 4.595
Epoch   6 Batch   24/46   train_loss = 4.423
Epoch   7 Batch   28/46   train_loss = 4.295
Epoch   8 Batch   32/46   train_loss = 4.430
Epoch   9 Batch   36/46   train_loss = 4.146
Epoch  10 Batch   40/46   train_loss = 4.115
Epoch  11 Batch   44/46   train_loss = 3.889
Epoch  13 Batch    2/46   train_loss = 3.986
Epoch  14 Batch    6/46   train_loss = 3.593
Epoch  15 Batch   10/46   train_loss = 3.490
Epoch  16 Batch   14/46   train_loss = 3.525
Epoch  17 Batch   18/46   train_loss = 3.313
Epoch  18 Batch   22/46   train_loss = 3.308
Epoch  19 Batch   26/46   train_loss = 3.190
Epoch  20 Batch   30/46   train_loss = 3.199
Epoch  21 Batch   34/46   train_loss = 3.161
Epoch  22 Batch   38/46   train_loss = 2.998
Epoch  23 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [3]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input_ = loaded_graph.get_tensor_by_name("input:0")
    initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state = loaded_graph.get_tensor_by_name("final_state:0")
    probabilities = loaded_graph.get_tensor_by_name("probs:0")
    return (input_, initial_state, final_state, probabilities)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [5]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    #print("P: ", probabilities)
    #print("V: ", int_to_vocab)
    best_guess_index = np.argmax(probabilities)
    return int_to_vocab[best_guess_index]



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [19]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer_simpson'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
homer_simpson:(reluctant) all right, i'm sorry...(bitter) but i will not save the rainforest.
peter_buck: good enough. let's eat. moe? i don't know.
moe_szyslak: nah, it's an american bank. i'm gonna good about a beer.
moe_szyslak:(to self) i need a feeling moe i got a woman. who's which they don't make a baby for me.
moe_szyslak: you know what i have. i've got a great job i ever heard?(takes out)
homer_simpson:(" the clincher") and i mean, i'm really talking about?
homer_simpson: well, i gotta pay for the money and the best thing--(laughs)
homer_simpson:(excited) oh, absolutely.
moe_szyslak: well, the people are a man who is drunk.
lenny_leonard:(to self) pass 'em a name.


moe_szyslak: eh, how would you like to tell you my buddy.
homer_simpson: i think i'll have to get... my dad's gonna take you in


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.